# Data Preprocessing - Improvement 1
This section focuses on cleaning, transforming, and preparing the e-commerce dataset for analysis and modeling in the first improvement of application analysis of FCPO in e-commerce domain.


In [83]:
import itertools
import pandas as pd
import numpy as np
import random
import csv
import time
import copy
import ast  
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

In [84]:
class DataPreprocessorEcommerce():
    def __init__(self, datapath, itempath):
        '''
        Load data from the E commerce Purchase Dataset
        List the users and the items
        List all the users historic
        '''
        self.data  = self.load_data(datapath, itempath)
        print("Og Data")
        print(self.og_data.head())
        print("Data")
        print(self.data.head())
        
        self.userId = np.array(self.data['userId'].values.tolist())
        self.itemId = np.array(self.data['itemId'].values.tolist())
        
        # Subtract 501 from each element in self.itemId
        self.itemId = self.itemId - 501
        
        self.data['userId'] = list(self.userId)
        self.data['itemId'] = list(self.itemId)
        
        print("after list")
        print(self.data.head())
        
        self.users = self.data['userId'].unique()   #list of all users
        self.items = self.data['itemId'].unique()   #list of all items
        self.nb_user = len(self.users)
        self.nb_item = len(self.items)
        print('total num of users:',self.nb_user)
        print('total num of items:',self.nb_item)
        
        #a list contains the rating history of each user
        self.histo = self.gen_histo()


    def load_data(self, dataname, datapath):
        '''
        Load the data
        '''
        # Read the CSV file without specifying column names or dtypes
        data = pd.read_csv(datapath)
        self.og_data = copy.deepcopy(data)#store original dataset
        
        # Define the desired column names and their corresponding dtypes
        desired_columns = ['customer_id', 'product_category', 'time_on_site [Minutes]', 'clicks_in_site', 'date']
        dtypes = {
            'customer_id': np.int32,
            'product_category': np.int32,
            'time_on_site [Minutes]': np.float64,
            'clicks_in_site': np.float64,
        }

        # Filter the DataFrame to keep only the desired columns
        data = data[desired_columns]
        #print(data.head())

        # Rename the columns (if needed) and set their dtypes
        data = data.astype(dtypes)
        #print(data.head())

        # Parse the date column and ensure dayfirst is True
        data['date'] = pd.to_datetime(data['date'], dayfirst=True)
        
        # Find the minimum date
        min_date = data['date'].min()

        # Calculate the difference in days
        data['days_since_min'] = (data['date'] - min_date).dt.days
        #print(data.head())
        # Rename columns to desired names
        data.rename(columns={
            'customer_id': 'userId',
            'product_category': 'itemId',
            'time_on_site [Minutes]': 'time_on_site',
            'clicks_in_site': 'clicks_in_site',
            'days_since_min': 'timestamp'
        }, inplace=True)
        #print(data.head())
        
        # Calculate 'rating' as clicks_in_site / time_on_site where both are non-null
        data['rating'] = np.where(data['clicks_in_site'].notna() & data['time_on_site'].notna(),
                                data['clicks_in_site'] / data['time_on_site'],
                                np.nan)
        
        # Drop the individual time_on_site and clicks_in_site columns
        data.drop(columns=['time_on_site', 'clicks_in_site','date'], inplace=True)

        return data

    def read_file(self, filename, isTrain=True):
        df = pd.read_csv(filename)
        user = df['user'].values.tolist()
        state = [ast.literal_eval(i) for i in df['state'].values.tolist()]
        if isTrain:
            state_reward = [ast.literal_eval(i) for i in df['state_reward'].values.tolist()]
        history = [np.array(ast.literal_eval(i)) for i in df['history'].values.tolist()]
        rewards = [ast.literal_eval(i) for i in df['rewards'].values.tolist()]
        data = pd.DataFrame ()
        data['user'] = user
        data['state'] = state
        if isTrain:
            data['state_reward'] = state_reward
        data['history'] = history
        data['rewards'] = rewards
        return data
    
    def gen_histo(self):
        '''
        Group all rates given by users and store them from older to most recent.

        Returns
        -------
        result :    List(DataFrame)
        List of the historic for each user
        '''
        print('start generating user history...')
        historic_users = []
        for i, u in tqdm(enumerate(self.users)):
            temp = self.data[self.data['userId'] == u]
            temp = temp.sort_values ('timestamp').reset_index ()
            temp.drop ('index', axis = 1, inplace = True)
            historic_users.append (temp)
        return historic_users


    def sample_histo_v6(self, user_histo, nb_states, nb_actions, pivot_rating=0):
        n = len(user_histo)
        states = []
        actions = []
        rewards = []
        states_prime = []
        done = []
        state_len = nb_states
        action_len = nb_actions
        
        item_list = user_histo['itemId'].values.tolist()
        click_list = user_histo['rating'].values.tolist()
        
        initial_state = []
        initial_end = 0
        for i in range(len(item_list)):
            if click_list[i] > pivot_rating and len(initial_state) < state_len:
                initial_state.append(item_list[i])
                initial_end = i
        if len(initial_state) == state_len and (initial_end + action_len <= len(item_list)):
            current_state = copy.copy(initial_state)
            for i in range(initial_end+1,len(item_list),action_len):
                if i+action_len <= len(item_list):
                    actions.append(item_list[i:i+action_len])
                    rewards.append(click_list[i:i+action_len])
                    states.append(copy.copy(current_state))
                    done.append(False)
                    for j in range(i,i+action_len):
                        if click_list[j] > pivot_rating:
                            current_state.append(item_list[j])
                            del current_state[0]
                    states_prime.append(copy.copy(current_state))
        if len(done) > 0:
            done[-1] = True
        return states, actions, rewards, states_prime, done

    def sample_histo_v5(self, user_histo, nb_states, pivot_rating):
        prop_histo = user_histo[user_histo['rating'] >= pivot_rating]
        if len(prop_histo) > nb_states:
            user = user_histo['userId'][0]
            initial_state =  prop_histo[0:nb_states]['itemId'].values.tolist()
            initial_rewards = prop_histo[0:nb_states]['rating'].values.tolist()
            user_history =  prop_histo[nb_states:]['itemId'].values.tolist()
            rewards = prop_histo[nb_states:]['rating'].values.tolist()
        return user, initial_state, initial_rewards, user_history, rewards
    
            
            
    def get_orginal_data(self, train_filename, test_filename):
        self.data.to_csv('/kaggle/working/train_data_org.csv', index=False, header=None)
        print('done!')
        
        #write the cost embedding 
        items = self.data['itemId'].to_list()
        items = np.array(items)
        idx, nb = np.unique(items, return_counts=True)
        # idx,nb
        #print(idx)
        #print(nb)
        pivot_point = 0
        while np.sum(nb>pivot_point)/self.nb_item >= 0.2:
            pivot_point += 1
        print('pivot point:', pivot_point)

        cost_indicator = np.zeros(self.nb_item)
        item_exposure = np.zeros(self.nb_item)
        for i in range(len(nb)):
            item_exposure[idx[i]] = nb[i]
            if nb[i] > pivot_point:
                cost_indicator[idx[i]] += 1
        np.save('/kaggle/working/item_cost_indicator_28_e_commerce.npy', cost_indicator)
        print(cost_indicator.shape)
        print(cost_indicator)
        np.save('/kaggle/working/item_exposure_28_e_commerce.npy', item_exposure)
        
                
                    
    def generate_pmf(self, output_path_item, output_path_user):
        # Convert to a sparse matrix
        user_ids = self.data['userId']
        item_ids = self.data['itemId']
        ratings = self.data['rating']

        # Adjust indices for 0-based indexing if necessary
        user_ids -= user_ids.min()
        item_ids -= item_ids.min()
        
        # Map user and item IDs to a 0-based index for the sparse matrix
        unique_users, user_index = np.unique(user_ids, return_inverse=True)
        unique_items, item_index = np.unique(item_ids, return_inverse=True)

        # Create the sparse user-item interaction matrix
        num_users = len(unique_users)
        num_items = len(unique_items)
        user_item_matrix = coo_matrix((ratings, (user_index, item_index)), shape=(num_users, num_items))

        # Apply Truncated SVD to approximate PMF
        embedding_dim = 10
        svd = TruncatedSVD(n_components=embedding_dim)

        # Get user and item embeddings
        user_embeddings = svd.fit_transform(user_item_matrix)    # User embeddings
        item_embeddings = svd.components_.T                      # Item embeddings

        # Convert embeddings to DataFrames for easy export
 
        item_embedding_df = pd.DataFrame(item_embeddings)
        user_embedding_df = pd.DataFrame(user_embeddings)

        # Save embeddings to CSV
        item_embedding_df.to_csv(output_path_item)
        user_embedding_df.to_csv(output_path_user)

        print(f"Item embeddings saved to {output_path_item}")
        print(item_embedding_df.shape)
        print(item_embedding_df)
        print(f"User embeddings saved to {output_path_user}")
        print(user_embedding_df.shape)
        print(user_embedding_df)

In [85]:
datapath = '/kaggle/input/e-commerce-purchase-dataset/purchase_data_exe.csv'
# print(datapath)
dg = DataPreprocessorEcommerce('e_commerce', datapath)

Og Data
         date  customer_id  product_category payment_method  value [USD]  \
0  20/11/2018        37077               505         credit        49.53   
1  20/11/2018        59173               509         paypal        50.61   
2  20/11/2018        41066               507         credit        85.99   
3  20/11/2018        50741               506         credit        34.60   
4  20/11/2018        53639               515         paypal       266.27   

   time_on_site [Minutes]  clicks_in_site  Unnamed: 7  
0                    12.0               8         NaN  
1                    25.9               8         NaN  
2                    34.9              11         NaN  
3                    16.5               9         NaN  
4                    43.1              30         NaN  
Data
   userId  itemId  timestamp    rating
0   37077     505          0  0.666667
1   59173     509          0  0.308880
2   41066     507          0  0.315186
3   50741     506          0  0.545455

19774it [00:28, 699.50it/s]


In [86]:
unique_items = np.unique(dg.itemId)
for x in unique_items:
    print(x)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [87]:
dg.histo[5]

,userId,itemId,timestamp,rating
0,39783,0,0,0.714286
1,39783,1,8,0.010657


In [88]:
train_filename = '/kaggle/working/train_data_e_commerce.csv'
test_filename = '/kaggle/working/test_data_e_commerce.csv'
validation_filename = '/kaggle/working/validation_data_e_commerce.csv'
train_test_ratio=0.8
pivot_rating=0
nb_states=1
nb_history = 2
nb_actions=1
users = []
initial_states = []
initial_rewards = []
user_histories = []
rewards = []

In [89]:
nb_cases = 0
for user_histo in dg.histo:
    try:
        prop_histo = user_histo[user_histo['rating'] >= pivot_rating]
        if len(prop_histo) >= nb_states:
            user = user_histo['userId'][0]
            init_state =  prop_histo[0:nb_states]['itemId'].values.tolist()
            init_r = prop_histo[0:nb_states]['rating'].values.tolist()
            u_history =  prop_histo[nb_states:]['itemId'].values.tolist()
            r = prop_histo[nb_states:]['rating'].values.tolist()
            #user, init_state, init_r, u_history, r = dg.sample_histo_v6(user_histo, nb_states, pivot_rating)
            # Print variables with a tab before each output
            #print(f"\tuser: {user}")
            #print(f"\tinit_state: {init_state}")
            #print(f"\tinit_r: {init_r}")
            #print(f"\tu_history: {u_history}")
            if len(u_history) >= nb_history:
                #print(f"\tr: {r}")
                #print(f"\tu_history: {u_history}")
                nb_cases += 1
                users.append(user)
                initial_states.append(init_state)
                initial_rewards.append(init_r)
                user_histories.append(u_history)
                rewards.append(r)
        continue
    except:
        continue
        
#print(f"\tuser: {users}")
#print(f"\tinit_state: {initial_states}")
#print(f"\tinit_r: {initial_rewards}")
#print(f"\tu_history: {user_histories}")
#print(f"\tr: {rewards}") 
nb_cases

660

In [90]:
data_to_split = pd.DataFrame()
data_to_split['user'] = users
data_to_split['state'] = initial_states
data_to_split['state_reward'] = initial_rewards
data_to_split['history'] = user_histories
data_to_split['rewards'] = rewards

In [91]:
from sklearn.model_selection import train_test_split
import pandas as pd
train_data, test_data = train_test_split(
        data_to_split, train_size=train_test_ratio, random_state=42
    )

In [92]:
print("train")
print(train_data.shape)
print(train_data)
print("test")
print(test_data.shape)
print(test_data)

train
(528, 5)
      user state           state_reward     history  \
18   38300  [12]  [0.37037037037037035]      [3, 0]   
363  27752   [5]    [1.417624521072797]  [8, 5, 10]   
597  42041   [4]   [0.8148148148148148]  [0, 2, 14]   
541  42523   [8]   [0.8421052631578947]     [10, 8]   
61   26766  [13]  [0.26022304832713755]     [14, 4]   
..     ...   ...                    ...         ...   
71   14183   [2]   [0.4225352112676057]   [4, 9, 9]   
106  16332  [14]  [0.18324607329842932]  [2, 10, 3]   
270  17239  [10]   [1.1188811188811187]    [12, 11]   
435  56299   [4]   [1.0738255033557047]     [12, 1]   
102  46781   [0]    [2.258064516129032]      [1, 3]   

                                               rewards  
18            [0.8588957055214723, 1.7777777777777777]  
363     [0.75, 0.5084745762711864, 1.0294117647058825]  
597  [4.509803921568627, 0.7894736842105263, 0.2550...  
541          [1.1475409836065573, 0.31088082901554404]  
61                        [1.5384615384

In [93]:
#print(len(train_data_rewards.shape[1]),len(test_data_rewards.shape[1]))
if train_filename != None and test_filename != None: 
    train_data.to_csv(train_filename, index=False)
    test_data.to_csv(validation_filename, index=False)
    test_data = test_data.drop(columns=['state_reward'])
    test_data.to_csv(test_filename, index=False)

In [94]:
dg.data

,userId,itemId,timestamp,rating
0,37077,4,0,0.666667
1,59173,8,0,0.308880
2,41066,6,0,0.315186
3,50741,5,0,0.545455
4,53639,14,0,0.696056
...,...,...,...,...
24994,33699,12,9,0.155211
24995,38652,0,9,0.180328
24996,30222,3,9,0.852273
24997,30183,6,9,1.305842


In [95]:
num_unique_customers = dg.og_data['customer_id'].nunique()
print("Number of unique items in customerId:", num_unique_customers)
num_unique_product = dg.og_data['product_category'].nunique()
print("Number of unique items in product_category:", num_unique_product)

Number of unique items in customerId: 19774
Number of unique items in product_category: 15


In [96]:
dg.histo[6]

,userId,itemId,timestamp,rating
0,26767,13,0,0.689655


# Generate item cost indicator

In [97]:
#write the cost embedding 
items = dg.data['itemId'].to_list()
items = np.array(items)
idx, nb = np.unique(items, return_counts=True)
# idx,nb
#print(idx)
#print(nb)
pivot_point = 0
while np.sum(nb>pivot_point)/dg.nb_item >= 0.2:
    pivot_point += 1
print('pivot point:', pivot_point)

cost_indicator = np.zeros(dg.nb_item)
item_exposure = np.zeros(dg.nb_item)
for i in range(len(nb)):
    item_exposure[idx[i]] = nb[i]
    if nb[i] > pivot_point:
        cost_indicator[idx[i]] += 1
        
cost_indicator_array =  cost_indicator.flatten()
np.savetxt('/kaggle/working/item_cost_indicator_28_e_commerce.csv', cost_indicator_array, delimiter=",")

#print(cost_indicator_array.shape)
print(cost_indicator_array)

pivot point: 1680
[0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Generate pmf embeddings

In [98]:
output_path_item = "/kaggle/working/pmf_item_embed_e_commerce.npy"
output_path_user = "/kaggle/working/pmf_user_embed_e_commerce.npy"
dg.generate_pmf(output_path_item,output_path_user)

Item embeddings saved to /kaggle/working/pmf_item_embed_e_commerce.npy
(15, 10)
           0         1         2         3         4         5         6  \
0   0.040239  0.166645  0.265771  0.779394 -0.480220 -0.158551 -0.170822   
1   0.022893  0.067520  0.051299  0.063114  0.027114  0.046764  0.178585   
2   0.986666 -0.130360 -0.037187 -0.045653 -0.032384 -0.019507 -0.036355   
3   0.029421  0.084209  0.054184  0.098990  0.045350  0.080620  0.219691   
4   0.084689  0.872785 -0.444161 -0.115791 -0.065977 -0.045554 -0.082166   
5   0.073868  0.074654  0.045524  0.081932  0.048164  0.055599  0.224709   
6   0.038864  0.123187  0.165310  0.135408  0.169822  0.909561 -0.255007   
7   0.028377  0.077111  0.102801  0.085937  0.057380  0.062122  0.731695   
8   0.030809  0.075206  0.056055  0.035284  0.006914  0.034877  0.148630   
9   0.020957  0.097573  0.051172  0.076103  0.031323  0.050281  0.228400   
10  0.039951  0.063347  0.041115  0.052846  0.036662  0.036850  0.102727   
11  0.05